In [2]:
# tester.ipynb
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))


from src.analyzer import analyze_speech

# Quick analysis
result = analyze_speech("../data/l2arctic_spontaneous/L2A_019.wav")
# result = analyze_speech("../samples/sample6.flac")

# Access results
print(f"Score: {result['verdict']['fluency_score']}")
print(f"Fillers detected: {len(result['filler_events'])}")

result

Analyzing audio: ../data/l2arctic_spontaneous/L2A_019.wav
Context: conversational

[1/5] Transcribing with Whisper (verbatim)...
  Duration: 74.88s
  Words: 101

[2/5] Marking filler words...
  Marked: 2 filler words
  Content words: 99

[3/5] Aligning words with WhisperX...
  Aligned: 101 words

[4/5] Detecting subtle fillers with Wav2Vec2...


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Total events: 12

[5/5] Calculating fluency score...

✓ Fluency Score: 42/100
✓ Readiness: not_ready
Score: 42
Fillers detected: 12


{'verdict': {'fluency_score': 42, 'readiness': 'not_ready'},
 'benchmarking': {'percentile': 30,
  'target_score': 80,
  'score_gap': 38,
  'estimated_guided_practice_hours': 22.8},
 'normalized_metrics': {'wpm': 79.32692307692308,
  'fillers_per_min': np.float64(2.884615384615385),
  'stutters_per_min': 0.0,
  'long_pauses_per_min': 1.6025641025641026,
  'very_long_pauses_per_min': 0.0,
  'pause_time_ratio': np.float64(0.06784188034188035),
  'pause_variability': np.float64(0.27122297978214244),
  'vocab_richness': 0.6363636363636364,
  'repetition_ratio': np.float64(0.10101010101010101)},
 'opinions': {'primary_issues': [{'issue': 'hesitation_structure',
    'severity': 'high',
    'root_cause': 'Pauses frequently interrupt sentence flow.',
    'score_impact': 12},
   {'issue': 'filler_dependency',
    'severity': 'high',
    'root_cause': 'Fillers replace silent planning pauses.',
    'score_impact': 12},
   {'issue': 'delivery_pacing',
    'severity': 'medium',
    'root_cause': 'S